In [ ]:
#hide
# from torchlife import *

# TorchLife

> Survival Analysis using pytorch

This library takes a deep learning approach to Survival Analysis.

## Install

`pip install torchlife`

## How to use

Fill me in please! Don't forget code examples: